In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# !pip install contractions
# import contractions
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


from absl import logging
import tensorflow as tf
import tensorflow_hub as hub

import re    #module for regular expression operations
import seaborn as sns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
Data = pd.read_csv('TrainData.csv')
Data['Sentences'] = Data['Sentences'].apply(lambda x: '<sos> '+x+' <eos>')
Data['Tags'] = Data['Tags'].apply(lambda x: '<tsos> '+x+' <teos>')
Data.head()

,Sentences,Tags
0,<sos> what movies star bruce willis <eos>,<tsos> O O O B-ACTOR I-ACTOR <teos>
1,<sos> show me films with drew barrymore from t...,<tsos> O O O O B-ACTOR I-ACTOR O O B-YEAR <teos>
2,<sos> what movies starred both al pacino and r...,<tsos> O O O O B-ACTOR I-ACTOR O B-ACTOR I-ACT...
3,<sos> find me all of the movies that starred h...,<tsos> O O O O O O O O B-ACTOR I-ACTOR O B-ACT...
4,<sos> find me a movie with a quote about baseb...,<tsos> O O O O O O O O O O O <teos>


In [ ]:
val_split = 0.2
N = Data.values.shape[0]
train_data = Data.values[int(N*val_split):,0:2]
test_data = pd.read_csv("./TestData.csv")
test_data['Sentences'] = test_data['Sentences'].apply(lambda x: '<sos> '+x+' <eos>')
test_data['Tags'] = test_data['Tags'].apply(lambda x: '<tsos> '+x+' <teos>')
test_data.head()

,Sentences,Tags
0,<sos> are there any good romantic comedies out...,<tsos> O O O O B-GENRE I-GENRE O B-YEAR I-YEAR...
1,<sos> show me a movie about cars that talk <eos>,<tsos> O O O O O B-PLOT I-PLOT I-PLOT <teos>
2,<sos> list the five star rated movies starring...,<tsos> O O B-RATINGS_AVERAGE I-RATINGS_AVERAGE...
3,<sos> what science fiction films have come out...,<tsos> O B-GENRE I-GENRE O O O O B-YEAR <teos>
4,<sos> did the same director make all of the ha...,<tsos> O O O O O O O O B-TITLE I-TITLE O <teos>


In [ ]:
test_data = test_data.values

In [ ]:
val_data = train_data[:int(N*val_split),0:2]

In [ ]:
print(val_data.shape[0]+train_data.shape[0],N,test_data.shape[0])

9775 9775 2443


In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(7820, 2)
(1955, 2)
(2443, 2)


In [ ]:
train_d , train_l = [i.split(' ') for i in train_data[:,0]], [i.split(' ') for i in train_data[:,1]]
val_d , val_l = [i.split(' ') for i in val_data[:,0]], [i.split(' ') for i in val_data[:,1]]
test_d , test_l = [i.split(' ') for i in test_data[:,0]], [i.split(' ') for i in test_data[:,1]]

In [ ]:
max_length = 32

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_d + val_d)
sequences = tokenizer.texts_to_sequences(train_d + val_d)

word_index = tokenizer.word_index
print("unique tokens except pad_token- "+str(len(word_index)))
vocab_size = len(tokenizer.word_index) + 1  #  with pad_token
print('vocab_size - '+str(vocab_size))

unique tokens except pad_token- 5971
vocab_size - 5972


In [ ]:
padded_lines = pad_sequences(sequences, maxlen=max_length, padding='post')

x_train_pad = padded_lines[0:-len(val_d)]
x_val_pad = padded_lines[-len(val_d):]
print(type(x_train_pad))

<class 'numpy.ndarray'>


In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_d)
x_test_pad = pad_sequences(test_sequences, maxlen=max_length, padding = 'post')
X_test_pad = []
test_L = []
for k,i in enumerate(x_test_pad):
  if np.all(i==0)==False:
    X_test_pad.append(i)
    test_L.append(test_l[k])

x_test_pad = np.array(X_test_pad)
# test_l = np.array(test_L)

# label_sequences = label_tokenizer.texts_to_sequences(test_L)
# y_test_pad = pad_sequences(label_sequences, maxlen=max_length, padding='post')

# print(type(y_test_pad))

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(train_l + val_l)
label_sequences = label_tokenizer.texts_to_sequences(train_l + val_l)

label_index = label_tokenizer.word_index
print("unique tokens except pad_token- "+str(len(label_index)))
label_vocab_size = len(label_tokenizer.word_index) + 1  #  pad_token at 0th index
print('vocab_size - '+str(label_vocab_size))
# label_index["<pad>"] = 0

padded_lines = pad_sequences(label_sequences, maxlen=max_length, padding='post')

y_train_pad = padded_lines[0:-len(val_d)]
y_val_pad = padded_lines[-len(val_d):]
print(type(y_train_pad))


label_sequences = label_tokenizer.texts_to_sequences(test_L)
y_test_pad = pad_sequences(label_sequences, maxlen=max_length, padding='post')
print(type(y_test_pad))

unique tokens except pad_token- 27
vocab_size - 28
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
label_tokenizer.word_index

{'<teos>': 3,
 '<tsos>': 2,
 'b-actor': 6,
 'b-character': 21,
 'b-director': 17,
 'b-genre': 4,
 'b-plot': 12,
 'b-rating': 13,
 'b-ratings_average': 15,
 'b-review': 24,
 'b-song': 23,
 'b-title': 10,
 'b-trailer': 26,
 'b-year': 8,
 'i-actor': 5,
 'i-character': 22,
 'i-director': 14,
 'i-genre': 19,
 'i-plot': 11,
 'i-rating': 18,
 'i-ratings_average': 16,
 'i-review': 25,
 'i-song': 20,
 'i-title': 7,
 'i-trailer': 27,
 'i-year': 9,
 'o': 1}

In [ ]:
import time
SEED = 1234

np.random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True    


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    

batch_size = 512
size = x_train_pad.shape[0]
d = int(size/batch_size)
X_train_b = np.array_split(x_train_pad,d)
y_train_b = np.array_split(y_train_pad,d)

class BahdanauAttention(nn.Module):
    """Implements Bahdanau (MLP) attention"""
    
    def __init__(self, hidden_size, key_size=None, query_size=None):
        super(BahdanauAttention, self).__init__()
        
        # We assume a bi-directional encoder so key_size is 2*hidden_size
        key_size = 2 * hidden_size if key_size is None else key_size
        query_size = hidden_size if query_size is None else query_size

        self.key_layer = nn.Linear(key_size, hidden_size, bias=False)
        self.query_layer = nn.Linear(query_size, hidden_size, bias=False)
        self.energy_layer = nn.Linear(hidden_size, 1, bias=False)
        
        # to store attention scores
        self.alphas = None
        
    def forward(self, query=None, key=None, value=None, mask=None):
        assert mask is not None, "mask is required"

        # We first project the query (the decoder state).
        # The projected keys (the encoder states) were already pre-computated.
        query = self.query_layer(query)
        proj_key = self.key_layer(key)
        # Calculate scores.
        scores = self.energy_layer(torch.tanh(query.unsqueeze(1) + proj_key))
        scores = scores.squeeze(2).unsqueeze(1)
        
        # Mask out invalid positions.
        # The mask marks valid positions so we invert it using `mask & 0`.

        
        scores.data.masked_fill_(mask == 0, -float('inf'))
        
        # Turn scores to probabilities.
        alphas = F.softmax(scores, dim=-1)
        self.alphas = alphas        
        
        # The context vector is the weighted sum of the values.
        context = torch.bmm(alphas, value)
        
        # context shape: [B, 1, 2D], alphas shape: [B, 1, M]
        return context, alphas

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout,bidirectional=True,batch_first=True)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        #embedded = [batch size, src len, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        #outputs = [batch size, src len, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        h1 = (hidden[0,:,:] + hidden[1,:,:])/2
        #h2 = (hidden[2,:,:] + hidden[3,:,:])/2
        #h = torch.stack((h1,h2),dim=0)
        c1 = (cell[0,:,:] + cell[1,:,:])/2
        #c2 = (cell[2,:,:] + cell[3,:,:])/2
        #c = torch.stack((c1,c2),dim=0)
        return outputs, c1.unsqueeze(0)
class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        # self.embedding = nn.Embedding(output_dim, emb_dim)
        input_dim = output_dim + hid_dim*2 #(*2 for bidirectioanl enc)
        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, dropout = dropout, batch_first=True)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input,attn, hidden, cell):
        
        #input = [batch size]
        #hidden = [batch size, n layers * n directions, hid dim]
        #cell = [batch size, n layers * n directions, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        
        #input = [batch size,]
        
        input_onehot =  torch.nn.functional.one_hot(input,num_classes=self.output_dim)

        #input_onehot = [batch_size,output_dim]

        # attn = [batch_size,hid_dim*2] (*2 for bidirectioanl enc)

        concat_input = torch.cat((input_onehot,attn),dim=1).unsqueeze(1)
        # concat_input = [batch_size,1,output_dim + hid_dim*2]
        output, (hidden, cell) = self.rnn(concat_input, (hidden, cell))
        
        #output = [batch size, seq len, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.attention = attention
        

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        #assert encoder.n_layers == decoder.n_layers, \
        #    "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [ batch size, src len]
        #trg = [ batch size, trg len]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        trg_hid_dim = self.decoder.hid_dim
        src_hid_dim = self.encoder.hid_dim*2  # *2 for bidirectional encoder
        n_layer = self.decoder.n_layers
        
        #tensor to store decoder outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        outputs_e, cell = self.encoder(src.long())
        hidden = torch.zeros((n_layer,batch_size,trg_hid_dim)).to(device)
        
        attn, alpha = self.attention(hidden.squeeze(0),outputs_e,outputs_e,src.unsqueeze(1))  # key and value are same for Bahdanau attention
        #first input to the decoder
        input = trg[:,0]
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input.long(),attn.squeeze(1), hidden, cell)
            attn, alpha = self.attention(hidden.squeeze(0),outputs_e,outputs_e,src.unsqueeze(1))
            #place predictions in a tensor holding predictions for each token
            outputs[:,t] = output
            #decide if we are going to use teacher forcing or not
            teacher_force = np.random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[:,t] if teacher_force else top1
        
        return outputs


INPUT_DIM = vocab_size
OUTPUT_DIM = label_vocab_size
ENC_EMB_DIM = 100
# DEC_EMB_DIM = 
HID_DIM = 32
N_LAYERS = 1
ENC_DROPOUT = 0.25
DEC_DROPOUT = 0.25



enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).to(device)
attention = BahdanauAttention(HID_DIM).to(device)
dec = Decoder(OUTPUT_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT).to(device)

model = Seq2Seq(enc, dec, attention, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
#model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(),lr=0.05)

TRG_PAD_IDX = 0

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

def train(model, X, y, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    n_samples = 0
    n_trg = 0
    correct = 0
    
    for src,trg in zip(X,y):
        
        src,trg = torch.from_numpy(src).to(device),torch.from_numpy(trg).to(device)
        
        optimizer.zero_grad()
        output = model(src, trg)
        #trg = [batch size, trg len]
        #output = [batch size, trg len, output dim]
        output_dim = output.shape[-1]
        
        output_1 = output.view(-1, output_dim)
        trg_1 = trg.view(-1)
        
        #trg_1 = [(trg len) * batch size]
        #output_1 = [(trg len) * batch size, output dim]

        mask = torch.logical_not(torch.eq(trg.long(), torch.tensor(0)))  # =>[batch, seq_Len], each element is a bool value, and if there is a pad in the sequence, the corresponding position of the mask is False
        mask = mask.type(torch.bool)
        
        pred = torch.argmax(output,2)
        
        pred = torch.masked_select(pred,mask)
        true = torch.masked_select(trg.long(),mask)
        n_trg += true.shape[0]

        correct += torch.sum(true==pred)
        
        loss = criterion(output_1, trg_1.long())
        
        loss.backward()
        
        #torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()*true.shape[0]
        n_samples += trg.shape[0]
    return epoch_loss / n_trg , correct.float()/n_trg

def evaluate(model, X, y, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
                                           
        src,trg = X, y
        
        src, trg = torch.from_numpy(np.array(src)).to(device),  torch.from_numpy(np.array(trg)).to(device)
                                           
        output = model(src, trg, 0) #turn off teacher forcing

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output_1 = output.view(-1, output_dim)
        trg_1 = trg.view(-1)

        #trg_1 = [(trg len - 1) * batch size]
        #output_1 = [(trg len - 1) * batch size, output dim]

        loss = criterion(output_1, trg_1.long())

        epoch_loss = loss.item()
        
        mask = torch.logical_not(torch.eq(trg.long(), torch.tensor(0)))  # =>[batch, seq_Len], each element is a bool value, and if there is a pad in the sequence, the corresponding position of the mask is False
        mask = mask.type(torch.bool)
        
        pred = torch.argmax(output,2)
        pred = torch.masked_select(pred,mask)
        true = torch.masked_select(trg.long(),mask)
        
        acc = torch.sum(true==pred).float()/true.shape[0]

    return epoch_loss, acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

N_EPOCHS = 100
CLIP = 1
val_losses = []
train_losses = []
val_accs = []
train_accs = []
best_valid_acc = 0
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss,train_acc = train(model, X_train_b, y_train_b, optimizer, criterion, CLIP)
    valid_loss,valid_acc = evaluate(model, x_val_pad, y_val_pad, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model, './seq2seq_attn_ner.sav')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | \t Train Acc: {train_acc: 3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | \t Valid Acc: {valid_acc: 3f}')

    val_losses.append(valid_loss)
    val_accs.append(valid_acc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    
model = torch.load('./seq2seq_attn_ner.sav')

test_loss,test_acc = evaluate(model, x_test_pad, y_test_pad, criterion)

print(f'| Test Loss: {test_loss:.3f} | \t Test Acc: {test_acc: 3f}')



plt.title('Loss vs Epochs')
plt.plot(val_losses,label='valid')
plt.plot(train_losses,label='train')
plt.legend()
plt.show()

plt.plot(val_accs,label='valid_acc')
plt.plot(train_accs,label='train_acc')
plt.legend()
plt.legend()
plt.show()


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


The model has 651,660 trainable parameters
Epoch: 01 | Time: 0m 12s
	Train Loss: 2.126 | 	 Train Acc:  0.499172
	 Val. Loss: 2.063 | 	 Valid Acc:  0.499643
| Test Loss: 2.021 | 	 Test Acc:  0.504702
Epoch: 02 | Time: 0m 12s
	Train Loss: 1.927 | 	 Train Acc:  0.512256
	 Val. Loss: 2.015 | 	 Valid Acc:  0.499691
| Test Loss: 1.986 | 	 Test Acc:  0.505074
Epoch: 03 | Time: 0m 12s
	Train Loss: 1.888 | 	 Train Acc:  0.522534
	 Val. Loss: 1.967 | 	 Valid Acc:  0.507677
| Test Loss: 1.955 | 	 Test Acc:  0.496820
Epoch: 04 | Time: 0m 12s
	Train Loss: 1.851 | 	 Train Acc:  0.534760
	 Val. Loss: 1.961 | 	 Valid Acc:  0.506631
| Test Loss: 1.949 | 	 Test Acc:  0.493742
Epoch: 05 | Time: 0m 12s
	Train Loss: 1.817 | 	 Train Acc:  0.541979
	 Val. Loss: 1.923 | 	 Valid Acc:  0.517707
| Test Loss: 1.922 | 	 Test Acc:  0.503484
Epoch: 06 | Time: 0m 12s
	Train Loss: 1.795 | 	 Train Acc:  0.541948
	 Val. Loss: 1.885 | 	 Valid Acc:  0.530256
| Test Loss: 1.879 | 	 Test Acc:  0.518976
Epoch: 07 | Time: 0m 